# STS Benchmark Datasets

### Preparation

Setup all required libraries

In [3]:
import logging
import re
import sys

import numpy as np
import pandas as pd

from fse import Average, SIF, uSIF, Vectors, FTVectors, CSplitIndexedList

from re import sub

from scipy.stats import pearsonr

from nltk import word_tokenize

logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

Next, we require the sentences from the STS benchmark dataset.

In [4]:
file= "../evaluation/sts-test.csv"
similarities, sent_a, sent_b = [], [], []
with open(file, "r") as f:
    for l in f:
        line = l.rstrip().split("\t")
        similarities.append(float(line[4]))
        sent_a.append(line[5])
        sent_b.append(line[6])
similarities = np.array(similarities)
assert len(similarities) == len(sent_a) == len(sent_b)
task_length = len(similarities)

for i, obj in enumerate(zip(similarities, sent_a, sent_b)):
    print(f"{i}\tSim: {obj[0].round(3):.1f}\t{obj[1]:40s}\t{obj[2]:40s}\t")
    if i == 4:
        break

0	Sim: 2.5	A girl is styling her hair.             	A girl is brushing her hair.            	
1	Sim: 3.6	A group of men play soccer on the beach.	A group of boys are playing soccer on the beach.	
2	Sim: 5.0	One woman is measuring another woman's ankle.	A woman measures another woman's ankle. 	
3	Sim: 4.2	A man is cutting up a cucumber.         	A man is slicing a cucumber.            	
4	Sim: 1.5	A man is playing a harp.                	A man is playing a keyboard.            	


Each of these sentence requires some preparation (i.e. tokenization) to be used in the core input formats.
To reproduce the results from the uSIF paper this part is taken from https://github.com/kawine/usif/blob/master/usif.py

In [5]:
not_punc = re.compile('.*[A-Za-z0-9].*')

def prep_token(token):
    t = token.lower().strip("';.:()").strip('"')
    t = 'not' if t == "n't" else t
    return re.split(r'[-]', t)

def prep_sentence(sentence):
    tokens = []
    for token in word_tokenize(sentence):
        if not_punc.match(token):
            tokens = tokens + prep_token(token)
    return tokens

Next we define the IndexedList object. The IndexedList extends the previously constructed sent_a and sent_b list together. We additionally provide a custom function "prep_sentence" which performs all the prepocessing for a single sentence. Therefore we need the extention **CSplitIndexedList**, which provides you the option to provide a custom split function

In [6]:
sentences = CSplitIndexedList(sent_a, sent_b, custom_split=prep_sentence)

The IndexedList returns the core object required for fse to train a sentence embedding: A tuple. This object constists of words (a list of strings) and its corresponding index. The latter is important if multiple cores access the input queue simultaneously. Thus it must be always provided. The index represents the row in the matrix where it can later be found.

In [7]:
sentences[0]

(['a', 'girl', 'is', 'styling', 'her', 'hair'], 0)

Note, that IndexedList does not convert the sentences inplace but only on calling the __getitem__ method in order to turn the sentence into a tuple. You can access the original sentence using

In [8]:
sentences.items[0]

'A girl is styling her hair.'

### Loading the models

It is required for us to load the models as BaseKeyedVectors or as an BaseWordEmbeddingsModel. For this notebook, I already converted the models to a BaseKeyedVectors instance and saved the corresponding instance on my external harddrive. You have to replicate these steps yourself, because getting all the files can be a bit difficult, as the total filesize is around 15 GB.

In [9]:
models, results = {}, {}

The following code performs a disk-to-ram training. Passing a path to __wv_mapfile_path__ will store the corresponding word vectors (wv) as a numpy memmap array. This is required, because loading all vectors into ram would would take up a lot of storage unecessary. The wv.vectors file will be replace by its memmap representation, which is why the next models do not require the wv_mapfile_path argument, as they access the same memmap object.

The lang_freq="en" induces the frequencies of the words according to the wordfreq package. This functionality allows you to work with pre-trained embeddings which don't come with frequency information. The method overwrites the counts in the glove.wv.vocab class, so that all further models also benefit from this induction.

In [10]:
model_names = [
    "fasttext-wiki-news-subwords-300",
    "glove-twitter-100",
    "glove-twitter-200",
    "glove-twitter-25",
    "glove-twitter-50",
    "glove-wiki-gigaword-100",
    "glove-wiki-gigaword-200",
    "glove-wiki-gigaword-300",
    "glove-wiki-gigaword-50",
    "paragram-25",
    "paragram-300-sl999",
    "paragram-300-ws353",
    "paranmt-300",
    "word2vec-google-news-300",
]

vectors = {}

for name in model_names:
    vectors[name] = Vectors.from_pretrained(name, mmap="r")
    
ft = "fasttext-crawl-subwords-300"
vectors[ft] = FTVectors.from_pretrained(ft, mmap="r")

2021-12-02 22:15:48,128 : MainThread : INFO : loading Vectors object from /home/oborchers/.cache/huggingface/hub/fse__fasttext-wiki-news-subwords-300.ef21870476cd93435d83140fcf6e7171b517e337/fasttext-wiki-news-subwords-300.model
2021-12-02 22:15:50,262 : MainThread : INFO : loading vectors from /home/oborchers/.cache/huggingface/hub/fse__fasttext-wiki-news-subwords-300.ef21870476cd93435d83140fcf6e7171b517e337/fasttext-wiki-news-subwords-300.model.vectors.npy with mmap=r
2021-12-02 22:15:50,264 : MainThread : INFO : setting ignored attribute vectors_norm to None
2021-12-02 22:15:50,265 : MainThread : INFO : loaded /home/oborchers/.cache/huggingface/hub/fse__fasttext-wiki-news-subwords-300.ef21870476cd93435d83140fcf6e7171b517e337/fasttext-wiki-news-subwords-300.model
2021-12-02 22:15:52,247 : MainThread : INFO : loading Vectors object from /home/oborchers/.cache/huggingface/hub/fse__glove-twitter-100.7ef1c21d9bf90598a0c618c041d9817e50250183/glove-twitter-100.model
2021-12-02 22:15:54,714

In [11]:
algos = {
    "CBOW" : (Average, {"lang_freq":"en"}),
    "SIF" : (SIF, {"lang_freq":"en", "components" : 10}),
    "uSIF": (uSIF, {"lang_freq":"en", "length": 11}),
}
import itertools
combinations = list(itertools.product(algos.keys(), vectors.keys()))

## Computation of the results for the STS benchmark

We are finally able to compute the STS benchmark values for all models.

In [12]:
# This function is used to compute the similarities between two sentences.
# Task length is the length of the sts dataset.
def compute_similarities(task_length, model):
    sims = []
    for i, j in zip(range(task_length), range(task_length, 2*task_length)):
        sims.append(model.sv.similarity(i,j))
    return sims

In [13]:
for algo, name in combinations:
    class_obj = algos[algo][0]
    args = algos[algo][1]
    vec = vectors[name]
    m = class_obj(vec, **args)
    m.train(sentences)
    r = pearsonr(similarities, compute_similarities(task_length, m))[0].round(4) * 100
    results[f"{algo} {name}"] = r
    print(algo, name, f"{r:2.2f}")

2021-12-02 22:16:59,259 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en
2021-12-02 22:17:00,296 : MainThread : INFO : scanning all indexed sentences and their word counts
2021-12-02 22:17:00,681 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2021-12-02 22:17:00,996 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 999999 vocabulary: 1151 MB (1 GB)
2021-12-02 22:17:00,998 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2021-12-02 22:17:01,056 : MainThread : INFO : begin training
2021-12-02 22:17:01,514 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-02 22:17:01,515 : MainThread : INFO : training on 2758 effective sentences with 27172 effective words took 0s with 6007 sentences/s
2021-12-02 22:17:01,550 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency

CBOW fasttext-wiki-news-subwords-300 26.08


2021-12-02 22:17:02,646 : MainThread : INFO : scanning all indexed sentences and their word counts
2021-12-02 22:17:02,960 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2021-12-02 22:17:03,414 : MainThread : INFO : estimated memory for 2758 sentences with 100 dimensions and 1193514 vocabulary: 460 MB (0 GB)
2021-12-02 22:17:03,415 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2021-12-02 22:17:03,449 : MainThread : INFO : begin training
2021-12-02 22:17:03,807 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-02 22:17:03,808 : MainThread : INFO : training on 2758 effective sentences with 26828 effective words took 0s with 7673 sentences/s
2021-12-02 22:17:03,836 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


CBOW glove-twitter-100 33.81


2021-12-02 22:17:04,847 : MainThread : INFO : scanning all indexed sentences and their word counts
2021-12-02 22:17:05,299 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2021-12-02 22:17:05,747 : MainThread : INFO : estimated memory for 2758 sentences with 200 dimensions and 1193514 vocabulary: 917 MB (0 GB)
2021-12-02 22:17:05,748 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2021-12-02 22:17:05,782 : MainThread : INFO : begin training
2021-12-02 22:17:06,130 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-02 22:17:06,131 : MainThread : INFO : training on 2758 effective sentences with 26828 effective words took 0s with 7903 sentences/s
2021-12-02 22:17:06,162 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


...

uSIF paranmt-300 79.02


2021-12-02 22:19:18,166 : MainThread : INFO : scanning all indexed sentences and their word counts
2021-12-02 22:19:18,499 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2021-12-02 22:19:19,550 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 3000000 vocabulary: 3447 MB (3 GB)
2021-12-02 22:19:19,551 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2021-12-02 22:19:19,562 : MainThread : INFO : pre-computing uSIF weights for 3000000 words
2021-12-02 22:19:27,275 : MainThread : INFO : begin training
2021-12-02 22:19:27,643 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-02 22:19:27,888 : MainThread : INFO : computing 5 principal components took 0s
2021-12-02 22:19:27,894 : MainThread : INFO : removing 5 principal components took 0s
2021-12-02 22:19:27,895 : MainThread : INFO : training on 2758 effective sentences with 23116 effective wor

uSIF word2vec-google-news-300 67.15


2021-12-02 22:19:29,638 : MainThread : INFO : scanning all indexed sentences and their word counts
2021-12-02 22:19:29,946 : MainThread : INFO : finished scanning 2758 sentences with an average length of 9 and 27528 total words
2021-12-02 22:19:30,677 : MainThread : INFO : estimated memory for 2758 sentences with 300 dimensions and 2000000 vocabulary: 6877 MB (6 GB)
2021-12-02 22:19:30,679 : MainThread : INFO : initializing sentence vectors for 2758 sentences
2021-12-02 22:19:30,701 : MainThread : INFO : pre-computing uSIF weights for 2000000 words
2021-12-02 22:19:35,907 : MainThread : INFO : begin training
2021-12-02 22:19:36,305 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2021-12-02 22:19:36,461 : MainThread : INFO : computing 5 principal components took 0s
2021-12-02 22:19:36,469 : MainThread : INFO : removing 5 principal components took 0s
2021-12-02 22:19:36,470 : MainThread : INFO : training on 2758 effective sentences with 27528 effective wor

uSIF fasttext-crawl-subwords-300 69.55


In [14]:
pd.DataFrame.from_dict(results, orient="index", columns=["Pearson"])

,Pearson
CBOW fasttext-wiki-news-subwords-300,26.08
CBOW glove-twitter-100,33.81
CBOW glove-twitter-200,34.94
CBOW glove-twitter-25,26.15
CBOW glove-twitter-50,30.78
CBOW glove-wiki-gigaword-100,38.12
CBOW glove-wiki-gigaword-200,42.40
CBOW glove-wiki-gigaword-300,44.46
CBOW glove-wiki-gigaword-50,37.47
CBOW paragram-25,40.13


If you closely study the values above you will find that:
- SIF-Glove is almost equivalent to the values reported in http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark
- CBOW-Paranmt is a little better than ParaNMT Word Avg. in https://www.aclweb.org/anthology/W18-3012
- uSIF-Paranmt is a little worse than ParaNMT+UP in https://www.aclweb.org/anthology/W18-3012
- uSIF-Paragram is a little worse than PSL+UP in https://www.aclweb.org/anthology/W18-3012

However, I guess those differences might arise due to differences in preprocessing. Too bad we didn't hit 80. If you have any ideas why those values don't match exactly, feel free to contact me anytime.